### Future Forecast data fields

    "location": {
        "name": "Rancho Cucamonga",
        "region": "California",
        "country": "USA",
        "lat": 34.1076,
        "lon": -117.59,
        "tz_id": "America/Los_Angeles",
        "localtime_epoch": 1737094984,
        "localtime": "2025-01-16 22:23"
    },
    "current": {
        "last_updated_epoch": 1737094500,
        "last_updated": "2025-01-16 22:15",
        "temp_c": 10.1,
        "temp_f": 50.2,
        "is_day": 0,
        "condition": {
        "text": "Partly cloudy",
        "icon": "//cdn.weatherapi.com/weather/64x64/night/116.png",
        "code": 1003
        },
        "wind_mph": 3.8,
        "wind_kph": 6.1,
        "wind_degree": 239,
        "wind_dir": "WSW",
        "pressure_mb": 1022,
        "pressure_in": 30.18,
        "precip_mm": 0,
        "precip_in": 0,
        "humidity": 63,
        "cloud": 25,
        "feelslike_c": 9.6,
        "feelslike_f": 49.2,
        "windchill_c": 10.1,
        "windchill_f": 50.2,
        "heatindex_c": 10.7,
        "heatindex_f": 51.2,
        "dewpoint_c": 7.6,
        "dewpoint_f": 45.6,
        "vis_km": 16,
        "vis_miles": 9,
        "uv": 0,
        "gust_mph": 6,
        "gust_kph": 9.6
    },

### History API Data Field

    "forecast": {
        "forecastday": [
        {
            "date": "2024-02-01",
            "date_epoch": 1706745600,
            "day": {
            "maxtemp_c": 12.2,
            "maxtemp_f": 54,
            "mintemp_c": 6.9,
            "mintemp_f": 44.4,
            "avgtemp_c": 9.8,
            "avgtemp_f": 49.7,
            "maxwind_mph": 8.5,
            "maxwind_kph": 13.7,
            "totalprecip_mm": 21.17,
            "totalprecip_in": 0.83,
            "totalsnow_cm": 0,
            "avgvis_km": 9.1,
            "avgvis_miles": 5,
            "avghumidity": 82,
            "daily_will_it_rain": 1,
            "daily_chance_of_rain": 100,
            "daily_will_it_snow": 0,
            "daily_chance_of_snow": 0,
            "condition": {
                "text": "Light rain shower",
                "icon": "//cdn.weatherapi.com/weather/64x64/day/353.png",
                "code": 1240
            },}
            "uv": 3
            },
            "astro": {
            "sunrise": "06:48 AM",
            "sunset": "05:21 PM",
            "moonrise": "11:56 PM",
            "moonset": "10:16 AM",
            "moon_phase": "Waning Gibbous",
            "moon_illumination": 66
            },]}


In [14]:
import requests
import pprint
import json
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()
pd.set_option("display.max_rows", None, "display.max_columns", None)

WEATHER_API = os.getenv("WEATHER_API")
history_url = "https://api.weatherapi.com/v1/history.json"
forecast_url = "https://api.weatherapi.com/v1/forecast.json"


# def history_retrieval():

dates = [f'2024-{n}-01' for n in range(2, 13)]
idx = 1
history_data = []

while idx < 2:
    with open("history_api.json", "a") as file:

        history_params = {
            "q": "91730",
            "dt": dates[idx - 1],
            "end_dt": dates[idx],
            "hour": 0,
            "key": WEATHER_API,
        }

        response_history = requests.get(history_url, params=history_params)
        history_data.append(response_history.json()['forecast']['forecastday'])
        # json.dump(response_history.json()['forecast']['forecastday'], file, indent=4)
        idx += 1


# with open("history_api.json", 'r') as file:
#     print(file)
#     history_data = pd.json_normalize(file)
    

pprint.pprint(history_data)
# forecast_params = {
#     "q": "91730",
#     "days": "14",
#     "hour": 0,
#     "key": WEATHER_API,
# }


# response_forecast = requests.get(forecast_url, params=forecast_params)


[[{'astro': {'moon_illumination': 66,
             'moon_phase': 'Waning Gibbous',
             'moonrise': '11:56 PM',
             'moonset': '10:16 AM',
             'sunrise': '06:48 AM',
             'sunset': '05:21 PM'},
   'date': '2024-02-01',
   'date_epoch': 1706745600,
   'day': {'avghumidity': 82,
           'avgtemp_c': 9.8,
           'avgtemp_f': 49.7,
           'avgvis_km': 9.1,
           'avgvis_miles': 5.0,
           'condition': {'code': 1240,
                         'icon': '//cdn.weatherapi.com/weather/64x64/day/353.png',
                         'text': 'Light rain shower'},
           'daily_chance_of_rain': 100,
           'daily_chance_of_snow': 0,
           'daily_will_it_rain': 1,
           'daily_will_it_snow': 0,
           'maxtemp_c': 12.2,
           'maxtemp_f': 54.0,
           'maxwind_kph': 13.7,
           'maxwind_mph': 8.5,
           'mintemp_c': 6.9,
           'mintemp_f': 44.4,
           'totalprecip_in': 0.83,
           'totalprecip_

In [3]:
list_history = response_history.json()["forecast"]["forecastday"]
# unpack_response = lambda *x: list(x)

# unpacked_history = unpack_response(*dict_history['forecast']['forecastday'])
# pprint.pprint(dict_history)

# for day in list_history:
#     pprint.pprint(
#         {
#             "date": day["date"],
#             "mintemp_f": day["day"]["mintemp_f"],
#             "maxtemp_f": day["day"]["maxtemp_f"],
#             "avgtemp_f": day["day"]["avgtemp_f"],
#             "maxwind_mph": day["day"]["maxwind_mph"],
#             "avgvis_miles": day["day"]["avgvis_miles"],
#             "avghumidity": day["day"]["avghumidity"],
#             "rained": day["day"]["daily_will_it_rain"],
#         },
#         sort_dicts=False,
#     )
    # break




['2024-2-01', '2024-3-01', '2024-4-01', '2024-5-01', '2024-6-01', '2024-7-01', '2024-8-01', '2024-9-01', '2024-10-01', '2024-11-01', '2024-12-01']
{'q': '91730', 'dt': '2024-2-01', 'end_dt': '2024-3-01', 'hour': 0, 'key': 'c883e590c49041c095273023251601'}
{'q': '91730', 'dt': '2024-3-01', 'end_dt': '2024-4-01', 'hour': 0, 'key': 'c883e590c49041c095273023251601'}
{'q': '91730', 'dt': '2024-4-01', 'end_dt': '2024-5-01', 'hour': 0, 'key': 'c883e590c49041c095273023251601'}
{'q': '91730', 'dt': '2024-5-01', 'end_dt': '2024-6-01', 'hour': 0, 'key': 'c883e590c49041c095273023251601'}
{'q': '91730', 'dt': '2024-6-01', 'end_dt': '2024-7-01', 'hour': 0, 'key': 'c883e590c49041c095273023251601'}
{'q': '91730', 'dt': '2024-7-01', 'end_dt': '2024-8-01', 'hour': 0, 'key': 'c883e590c49041c095273023251601'}
{'q': '91730', 'dt': '2024-8-01', 'end_dt': '2024-9-01', 'hour': 0, 'key': 'c883e590c49041c095273023251601'}
{'q': '91730', 'dt': '2024-9-01', 'end_dt': '2024-10-01', 'hour': 0, 'key': 'c883e590c4904

In [ ]:
# import pandas as pd
# import sklearn

# df_history = pd.js
